In [ ]:
# 필요한 패키지 설치
%pip install chromadb llama_index.vector_stores.chroma llama-index-llms-gemini

In [1]:
# 라마인덱스 패키지 임포트
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.gemini import Gemini

from dotenv import load_dotenv
import os

c:\Code\LlamaIndex_Code\LlamaIndex_VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Gemini API 키 설정
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [3]:
# LLM 설정(OpenAI)
Settings.llm = OpenAI(
    temperature=0.5,
    model="gpt-4o",
    max_tokens=200,
    context_window=4096,
)

# # LLM 설정(Gemini)
# Settings.llm = Gemini(
#     temperature=0.5,
#     model="models/gemini-2.0-flash",
#     max_tokens=200,
#     context_window=4096,
# )

In [4]:
# 데이터 로드 (data가 있는 경로 입력)
documents = SimpleDirectoryReader("../data/pdf_sample2").load_data()

In [5]:
# 벡터 DB 생성 및 저장
db = chromadb.PersistentClient(path="../data/chroma_db")

# 컬렉션 생성 및 저장
chroma_collection = db.get_or_create_collection("quickstart")

2025-09-21 11:15:50,065 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [6]:
# ChromaDB를 LlamaIndex의 인덱싱 및 검색 파이프라인에 통합
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
# 문서 내용을 벡터 DB에 저장
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

2025-09-21 11:15:56,495 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [8]:
# 쿼리 엔진 생성
query_engine = index.as_query_engine()

# 쿼리 실행
query = "이 논문에서 제안하는 모델의 장점은 무엇인가요?"
response = query_engine.query(query)

# 응답 출력
print("\n질문:", query)
print("답변:", response)

2025-09-21 11:16:16,042 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-21 11:16:18,537 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



질문: 이 논문에서 제안하는 모델의 장점은 무엇인가요?
답변: 이 논문에서 제안하는 모델의 장점은 주로 효율성과 성능 향상에 있습니다. 특히, 이 모델은 번역 작업에서의 계산 복잡성을 줄이며, 병렬화가 용이하여 학습과 추론 속도가 빠릅니다. 또한, 복잡한 의존 관계를 효과적으로 처리할 수 있어 번역 품질이 개선됩니다.
